## Imports

In [ ]:
import numpy as np
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix
import time

## Preparing Matrices

In [ ]:
# the number of features
n = 61188
#number of classes
K = 20
#training instances
M = 12000
Normalize_X = False

#this will hold the documents that get mixed togerher!
confusion_matrix = np.zeros(shape=(21, 21), dtype=np.int16)

training_labels = np.load('../res/lr_labels.npy')
#loading the normalized training data!
training_data_sparse = sparse.load_npz('../res/lr_training_data.npz')
if Normalize_X:
    training_data_sparse = training_data_sparse.multiply(training_data_sparse.sum(axis=0)).tocsr()
XT = training_data_sparse.transpose()
weights_matrix = np.random.rand(K, n + 1)
# weights_sparse is the W matrix in the pdf
weights_sparse = sparse.csr_matrix(weights_matrix, dtype=np.float)
print('weights shape:', weights_sparse.shape)
print('training data shape:', training_data_sparse.shape)
print('training data transpose shape:', XT.shape)


delta = lil_matrix(np.zeros((K, M)), dtype = np.int16)
for i, label in enumerate(training_labels):
    delta[label - 1, i] = 1
delta = delta.tocsr()
print('delta shape:', delta.shape)
# you see the index 1 mapping here: we need to be careful!
#how to get the true class of a training examples
print(delta[:,0].nonzero()[0][0] + 1)
print(training_labels[0])
print(sparse.issparse(delta))

## LOAD WEIGHT MATRIX IF YOU WANT!

In [ ]:
def load_weight_sparse(file_name):
    global weights_sparse
    weights_sparse = sparse.load_npz(file_name)
# example : 
#load_weight_sparse("../res/weights_1000_runs_acc_85.npz")

## Begin Training

In [ ]:
ETHA = 0.005
LAMBDA = 0.01
EPOCH = 3000

t0 = time.time()
print("Started at:", t0)
for i in range(EPOCH):
    # print(weights_sparse)    
    pxy = weights_sparse * XT
    #normalize before exp
    pxy = pxy.multiply(1/pxy.sum(axis=0)).tocsr()
    pxy = pxy.expm1().tocsr()
    pxy = pxy.multiply(1/pxy.sum(axis=0)).tocsr()
    diff_exp = delta - pxy
    new_w = weights_sparse + ETHA * (diff_exp * training_data_sparse - weights_sparse.multiply(LAMBDA))
    weights_sparse = new_w
    if i % 100 == 0:
        print(f"epoch {i}")
    i += 1
    # np.nan_to_num(weights_sparse, copy=False)

t1 = time.time()
print('Finished at:', t1, "\nTotal runtime:", t1 - t0)

## Save Weights if you want

In [ ]:
## SAVE WEIGHTS
def save_weight(name=""):
    if not name:
        import datetime
        name = "../res/weight_matrix" + str(datetime.datatime.now())
    else:
        name = "../res/" + name
    sparse.save_npz(name, weights_sparse)
save_weight("weights_1000_runs_acc_85")

In [ ]:
print(weights_sparse.shape)                if actual < predicted:
                    min_idx = 
print(weights_sparse)

## Testing

In [ ]:
def test_weights(predictions, is_training=False):
    correct = 0
    predictions = predictions.multiply(1/predictions.sum(axis=0))
    predictions = predictions.expm1()
    predictions = predictions.multiply(1/predictions.sum(axis=0))
    predictions = predictions.todense()
    if is_training:
        for i in range(predictions.shape[1]):
            predicted = np.argmax(predictions[:,i]) + 1
            actual = training_labels[i]
            if predicted == actual:
                correct += 1
            else:
                #doing the confusion matrix
                min_idx = predicted
                #the row says the real class the column says what class it was confused wth
                confusion_matrix[actual, predicted] += 1
        print(f"Correct is {correct} and accuracy is {correct * 100/predictions.shape[1]}%")
        print("confusion matrix:")
        print(confusion_matrix)
    else:
        print('writing to file...')
        with open('../results/lr_out.csv', 'w') as out_stream:
            out_stream.write("id,class\n")
            ids = range(12001, 18775)
            for i in range(predictions.shape[1]):
                predicted_label = np.argmax(predictions[:,i]) + 1
                doc_id = ids[i]
                out_stream.write(f"{doc_id},{predicted_label}\n")
    print("Finished")

### training accuracy

In [ ]:
training_predictions = (weights_sparse * XT)
test_weights(training_predictions, is_training=True)

### training accuracy

In [ ]:
testing_data_sparse = sparse.load_npz('../res/lr_testing_data.npz')
test_XT = testing_data_sparse.transpose()
test_weights(weights_sparse * test_XT)




print('weights shape:', weights_sparse.shape)
print('testing data shape:', testing_data_sparse.shape)
print('testing data first column:', testing_data_sparse[:,0])
print('training data first column:', training_data_sparse[:,0])
# print('testing data transpose shape:', test_XT.shape)
# testing_predictions = (weights_sparse * test_XT).todense()
# test_weights(testing_predictions, is_training=False)